In [22]:
import numpy as np
import torch as T
import scipy; import scipy.optimize;
import time;

import matplotlib;
import matplotlib.pyplot as plt

import  forward_method as FM;


In [23]:
"""
cell 1: define
define distance between field and reflector;
define the smapling points.
"""
D=90*10**3; # distance between source and reflector
Nu=Nv=11;   #sampling points of far-/near-field
noise=0.5;   # rms of the noise added to measurement pattern.
rms_surface=0.08#mm # rms of the surface error.

Nx=5;    # sampling poitns on x direction; for panels
Ny=5;    # sampling points on y direction;
##############################################################################
List=np.array([[-500,500,-1500,-500,500,1500,-1500,-500,500,1500,-500,500],
               [1500,1500,500,500,500,500,-500,-500,-500,-500,-1500,-1500]]);
size=1000; # size of panel
Nx=5;    # sampling poitns on x direction;
Ny=5;    # sampling points on y direction;
f=100*10**3;

Lambda=1#mm
sigma=1200;Amp_0=1
# define u v range

u=np.linspace(-1,1,Nu)*0.002;
v=np.linspace(-1,1,Nu)*0.002;
gird=np.moveaxis(np.meshgrid(u,v),0,-1);
u=gird[...,0];
v=gird[...,1];
m_x,m_y,m_z=FM.reflector(List,size,Nx,Ny,f)
far_field=FM.get_function(List,size,Nx,Ny,f,u,v,f,Lambda,sigma,Amp_0);

# near field
near_field=FM.get_function(List,size,Nx,Ny,f,u,v,D,Lambda,sigma,Amp_0);

In [24]:
"""
Cell 2: get the measurement data!
"""
# get measurement data;
ad1=np.random.normal(0,rms_surface,(3,12)) # ad1 is the three different adjusters

print('adjusters for test data:');
print(ad1)
p=T.tensor(FM.adjuster2P(ad1));
real_meas,imag_meas=near_field(p);

# add noise to measurement data;
if noise !=0:
    FM.addnoise(real_meas,imag_meas,0,noise);

Data_meas=T.cat((real_meas,imag_meas));


adjusters for test data:
[[-0.017676   -0.11660406  0.15004756  0.02985367 -0.02726581 -0.04923315
   0.07648087  0.08722271  0.05483345 -0.01309926 -0.05728369 -0.01763052]
 [-0.12764133  0.07156887  0.03231274  0.08823446 -0.13266041  0.06919138
  -0.00557582  0.03003395 -0.0994673   0.12648421  0.08757917 -0.0460866 ]
 [-0.06659225 -0.13164274  0.0789126   0.18761358 -0.01187772 -0.03575967
  -0.15265021 -0.06516927  0.03825771  0.01692558  0.12100253 -0.06719387]]


In [25]:
"""
Cell 3:
fitting process with pyTorch package!
"""
print('>>>> fitting process with pyTorch package')
def fitfn(ad):
    p=FM.adjuster2P(ad);
    
    real,imag=near_field(p);
    r=T.cat((real_meas-real,imag_meas-imag))

    return r;
def fitfn0(ad):
    ad=T.tensor(ad, requires_grad=True);
    r=(fitfn(ad)**2).sum();
    r.backward();
    #print(r)
    return r.data.cpu().numpy(), ad.grad.data.cpu().numpy()

# define the initial adjusters position
ad=np.zeros((3,12));
start=time.clock();
x=scipy.optimize.minimize(fitfn0,ad,method="BFGS",jac=True,tol=1e-5);
elapsed = (time.clock() - start);
print("Time used:",elapsed);

print('1.fitting results: adjusters')
print(x.x.reshape(3,-1))
print('2.residual is %e'% x.fun)

error=(np.sum(np.abs(ad1-x.x.reshape(3,-1)))/36)*1000
print('3. rms of fitting adjusters positon relative to measurement adjusters: %e um'% error);

>>>> fitting process with pyTorch package
Time used: 308.42132461542553
1.fitting results: adjusters
[[-0.01607655 -0.11518959  0.15199692  0.0300363  -0.0271714  -0.05047187
   0.07574656  0.08735849  0.05465722 -0.01289772 -0.05704211 -0.01733283]
 [-0.12770233  0.07225611  0.02975199  0.08828042 -0.13272053  0.06815467
  -0.00491817  0.03012128 -0.09986492  0.1264089   0.08762026 -0.04584487]
 [-0.06491942 -0.12873741  0.07875068  0.18779418 -0.01180938 -0.03578596
  -0.15164391 -0.06488951  0.03880528  0.01642978  0.12106382 -0.06738681]]
2.residual is 1.300425e+04
3. rms of fitting adjusters positon relative to measurement adjusters: 6.060458e-01 um


In [18]:
"""
Cell 4
fitting process with numpy
"""
print('>>>>fitting process with numpy')
def fitfn(ad): 
    p=FM.adjuster2P(ad);
    real,imag=near_field(p);
    r=T.cat((real_meas-real,imag_meas-imag))
    return r;

def fitfn0(ad):
    ad=T.tensor(ad, requires_grad=True);
    r=fitfn(ad)
    return r.data.cpu().numpy()#, p.grad.data.cpu().numpy()

# define the initial adjusters position
ad=np.zeros((3,12));
start=time.clock();
x=scipy.optimize.leastsq(fitfn0,ad);
elapsed = (time.clock() - start);
print("Time used:",elapsed);

print('1.fitting results: adjusters')
print(x[0].reshape(3,-1))
error=(np.sum(np.abs(ad1-x[0].reshape(3,-1)))/36)*1000
print('2. surface error of fitting adjusters positon relative to measurement adjusters: %e um'% error);

>>>>fitting process with numpy
Time used: 20.501895975589235
1.fitting results: adjusters
[[ 0.17159607 -0.04345302 -0.04904771  0.08839964 -0.08877105 -0.02323882
  -0.10348874  0.2165223   0.00567226  0.00246854  0.19745818  0.01074936]
 [ 0.07633944  0.07355157 -0.09650522 -0.01163894 -0.04131937  0.01862301
   0.08204397  0.05447957  0.16530312 -0.02376186  0.07866487 -0.06984263]
 [-0.00975745  0.061757   -0.06186337  0.03205627  0.09275948 -0.02868077
  -0.04960083  0.00607885  0.11070863  0.06645419  0.00077842 -0.14530495]]
2. surface error of fitting adjusters positon relative to measurement adjusters: 2.079299e+00 um
